In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
import numpy as np
import matplotlib.pyplot as plt

In [52]:
(x_train, y_train),(x_test, y_test) = datasets.mnist.load_data()

In [53]:
x_train = 2*tf.convert_to_tensor(x_train, dtype=tf.float32)/255.-1
x_test = 2*tf.convert_to_tensor(x_test, dtype=tf.float32)/255.-1

In [54]:
y_train = tf.cast(y_train, dtype=tf.int32)
y_test = tf.cast(y_test, dtype = tf.int32)
y_train, y_test

(<tf.Tensor: shape=(60000,), dtype=int32, numpy=array([5, 0, 4, ..., 5, 6, 8], dtype=int32)>,
 <tf.Tensor: shape=(10000,), dtype=int32, numpy=array([7, 2, 1, ..., 4, 5, 6], dtype=int32)>)

In [55]:
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

In [56]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(512)
x_train.shape
x_test.shape

TensorShape([10000, 28, 28])

In [57]:
model = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [58]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.003)

In [61]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [60]:
for epoch in range(20):
  for batch_x, batch_y in train_dataset:
    with tf.GradientTape() as tape:
      batch_x = tf.reshape(batch_x, (-1,28*28))
      out = model(batch_x)
      loss = loss_fn(batch_y, out)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
  print(f"Epoch : {epoch+1}, Loss: {loss.numpy():.4f}")

Epoch : 1, Loss: 0.3104
Epoch : 2, Loss: 0.2667
Epoch : 3, Loss: 0.2299
Epoch : 4, Loss: 0.1915
Epoch : 5, Loss: 0.1451
Epoch : 6, Loss: 0.1607
Epoch : 7, Loss: 0.1062
Epoch : 8, Loss: 0.0814
Epoch : 9, Loss: 0.0374
Epoch : 10, Loss: 0.0686
Epoch : 11, Loss: 0.0710
Epoch : 12, Loss: 0.0088
Epoch : 13, Loss: 0.0113
Epoch : 14, Loss: 0.0319
Epoch : 15, Loss: 0.0142
Epoch : 16, Loss: 0.0472
Epoch : 17, Loss: 0.0007
Epoch : 18, Loss: 0.0073
Epoch : 19, Loss: 0.0004
Epoch : 20, Loss: 0.0122


In [63]:
x_test_flattened = tf.reshape(x_test, (-1,28*28))
test_loss, test_Acc = model.evaluate(x_test_flattened, y_test)
print(f"test_loss: {test_loss}")
print(f"test_Acc: {test_Acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9567 - loss: 0.1930
test_loss: 0.1461333930492401
test_Acc: 0.9656999707221985


In [64]:
y_test_np = y_test.numpy()

In [70]:
def predict_digit(index):
  sample_image = x_test[index]
  actual_label = np.argmax(y_test_np[index])
  sample_image_flattened = tf.reshape(sample_image, (1,28*28))
  prediction = model(sample_image_flattened).numpy()
  predicted_label = np.argmax(prediction)
  return prediction, actual_label, predicted_label


In [71]:
prediction, actual_label, predicted_label = predict_digit(0)
confidence = np.max(prediction)*100
print(f"Actual label: {actual_label}")
print(f"Predicted label: {predicted_label}")
print(f"Confidence: {confidence:.2f}%")

Actual label: 7
Predicted label: 7
Confidence: 100.00%


In [72]:
while True:
  try:
    index = int(input("Enter the index of the sample image: "))
    prediction, actual_label, predicted_label = predict_digit(index)
    confidence = np.max(prediction)*100
    print(f"Actual label: {actual_label}")
    print(f"Predicted label: {predicted_label}")
    print(f"Confidence: {confidence:.2f}%")
  except ValueError:
    print("Invalid input. Please enter an integer index.")
  except IndexError:
    print("Invalid index. Please enter a valid index.")

Enter the index of the sample image: 10
Actual label: 0
Predicted label: 0
Confidence: 100.00%
Enter the index of the sample image: 5
Actual label: 1
Predicted label: 1
Confidence: 99.92%
Enter the index of the sample image: 9
Actual label: 9
Predicted label: 9
Confidence: 100.00%
Enter the index of the sample image: 7
Actual label: 9
Predicted label: 9
Confidence: 100.00%
Enter the index of the sample image: -1
Actual label: 6
Predicted label: 6
Confidence: 100.00%
Enter the index of the sample image: ll
Invalid input. Please enter an integer index.


KeyboardInterrupt: Interrupted by user